## Import packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r "/content/drive/My Drive/data/arrays" ./arrays

In [4]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data.dataset import TensorDataset, Dataset
from torch.utils.data import DataLoader
import torch.optim as optim

import numpy as np
import logging
import os
import time
import sys
import json


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Model

### LSTM

In [0]:
class LSTM(nn.Module):
    def __init__(self, inputDim, hiddenNum, outputDim, seq_len=90, output_len=1, layerNum=3, drop_out=0.):

        super(LSTM, self).__init__()
        # hidden cell numbers
        self.hiddenNum = hiddenNum
        # input dimension
        self.inputDim = inputDim
        # output dimension
        self.outputDim = outputDim
        # layer number
        self.layerNum = layerNum
        # sequence length
        self.seq_len = seq_len
        # output length
        self.output_len = output_len

        # LSTM cell
        self.lstm = nn.LSTM(input_size=self.inputDim, hidden_size=self.hiddenNum,
                            num_layers=self.layerNum, dropout=drop_out,
                            batch_first=True)
        self.dropout = nn.Dropout(drop_out)
        self.layer_norm = nn.LayerNorm(hiddenNum)

        self.fc = nn.Linear(self.hiddenNum, self.outputDim)
        self.final_fc = nn.Linear(self.seq_len, self.output_len)

    def forward(self, x):
        # x = [batch, input_len, output_len]
        batch_size = x.size(0)
        h0 = torch.zeros(self.layerNum * 1, batch_size, self.hiddenNum)
        c0 = torch.zeros(self.layerNum * 1, batch_size, self.hiddenNum)
        h0, c0 = h0.to(device), c0.to(device)

        # output = [batch_size, seq_len, hidden_num]
        # hn = ([num_layer, batch_len, hidden_num],
        #       [num_layer, batch_len, hidden_num])
        output, hn = self.lstm(x, (h0, c0))
        output = self.layer_norm(output)

        #         output = output.view(output.size(0)*output.size(1), output.size(2))
        fc_output = self.fc(output)
        fc_output = self.dropout(fc_output)
        fc_output = fc_output.squeeze()
        fc_output = self.final_fc(fc_output)
        
        # fc_output = self.fc(output[:, -1, :])

        return fc_output

    def weight_init(self):
        for param in self.lstm.parameters():
            if len(param.shape) >= 2:
                nn.init.orthogonal_(param.data)
            else:
                nn.init.normal_(param.data)

### Attention-LSTM

In [0]:
class AttenLSTM(nn.Module):
    def __init__(self, inputDim, hiddenNum, outputDim, seq_len=90, output_len=1, layerNum=3, drop_out=0.):

        super(AttenLSTM, self).__init__()
        # hidden cell numbers
        self.hiddenNum = hiddenNum
        # input dimension
        self.inputDim = inputDim
        # output dimension
        self.outputDim = outputDim
        # layer number
        self.layerNum = layerNum
        # sequence length
        self.seq_len = seq_len
        # output length
        self.output_len = output_len

        # LSTM cell
        self.lstm = nn.LSTM(input_size=self.inputDim, hidden_size=self.hiddenNum,
                            num_layers=self.layerNum, batch_first=True)
        self.atten = ScaledDotProductAttention(dropout=drop_out)

        self.lstm_decoder = nn.LSTM(input_size=self.hiddenNum, hidden_size=self.hiddenNum,
                                    num_layers=self.layerNum, batch_first=True)
        
        self.dropout = nn.Dropout(drop_out)
        self.fc = nn.Linear(self.hiddenNum, self.outputDim)
        self.final_fc = nn.Linear(self.seq_len, self.output_len)
        self.weight_init()

    def forward(self, x):
        # x = [batch, input_len, output_len]
        batch_size = x.size(0)
        h0 = torch.zeros(self.layerNum * 1, batch_size, self.hiddenNum)
        c0 = torch.zeros(self.layerNum * 1, batch_size, self.hiddenNum)
        h0, c0 = h0.to(device), c0.to(device)

        # output = [batch_size, seq_len, hidden_num]
        # hn = ([num_layer, batch_len, hidden_num],
        #       [num_layer, batch_len, hidden_num])
        output, hn = self.lstm(x, (h0, c0))

        atten_output, _ = self.atten(output, output, output)
        # print(f"atten_output size: {atten_output.size()}")

        decoder_output, hn_d = self.lstm_decoder(atten_output, hn)

        # fc_output = F.relu(self.fc(atten_output))
        fc_output = self.fc(decoder_output)
        fc_output = fc_output.squeeze()
        fc_output = self.final_fc(fc_output)

        return fc_output

    def weight_init(self):
        for param in self.lstm.parameters():
            if len(param.shape) >= 2:
                nn.init.orthogonal_(param.data)
            else:
                nn.init.normal_(param.data)



class ScaledDotProductAttention(nn.Module):
    def __init__(self, dropout):
        super(ScaledDotProductAttention, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, scale=None):
        attention = torch.bmm(q, k.transpose(1, 2))
        if scale:
            attention = attention * scale
        # calculate softmax
        attention = self.softmax(attention)
        # add dropout
        attention = self.dropout(attention)
        # context
        context = torch.bmm(attention, v)
        return context, attention

class PositionalWiseFeedForward(nn.Module):

    def __init__(self, model_dim=512, ffn_dim=2048, dropout=0.0):
        super(PositionalWiseFeedForward, self).__init__()
        self.w1 = nn.Conv1d(model_dim, ffn_dim, 1)
        self.w2 = nn.Conv1d(ffn_dim, model_dim, 1)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(model_dim)

    def forward(self, x):
        output = x.transpose(1, 2)
        output = self.w2(F.tanh(self.w1(output)))
        output = self.dropout(output.transpose(1, 2))

        # add residual and norm layer
        output = self.layer_norm(x + output)
        return output

### TCN

In [0]:
from torch.nn.utils import weight_norm

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, input_len, output_len,
                 kernel_size=3, dropout=0.3, feature=None):
        super(TCN, self).__init__()

        # TCN nets
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout=dropout)

        # set features
        self.feature = feature

        self.decoder = nn.Linear(num_channels[-1], output_size)
        self.final_fc = nn.Linear(input_len, output_len)
        self.dropout = nn.Dropout(dropout)

        self.init_weights()

    def init_weights(self):
        # self.encoder.weight.data.normal_(0, 0.01)
        #         nn.init.xavier_normal_(self.tcn)
        nn.init.xavier_normal_(self.final_fc.weight.data)
        nn.init.normal_(self.final_fc.bias.data)

    def forward(self, x):
        """Input ought to have dimension (N, C_in, L_in), where L_in is the seq_len; here the input is (N, L, C)"""
        # emb = self.drop(input)
        if self.feature == 'pri':
            x = x[:, :, :1]
        elif self.feature == 'vol':
            x = x[:, :, -1].unsqueeze(2)
        y = self.tcn(x.transpose(1, 2)).transpose(1, 2)
        # print(f"price_y shape: {price_y.size()}; volume_y shape: {volume_y.size()}")
        # print(f"y shape: {y.size()}")
        # y = self.tcn(x.transpose(1, 2)).transpose(1, 2)
        y = self.decoder(y)
        y = self.final_fc(y[:, :, 0])
        # print("y.size: ", y.size())
        return self.dropout(y)


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        # init conv1
        nn.init.xavier_uniform_(self.conv1.weight, gain=np.sqrt(2))
        # nn.init.normal_(self.conv1.weight.data)
        if self.conv1.bias is not None:
            nn.init.normal_(self.conv1.bias.data)
        # init conv2
        nn.init.xavier_uniform_(self.conv2.weight, gain=np.sqrt(2))
        # nn.init.normal_(self.conv2.weight.data)
        if self.conv2.bias is not None:
            nn.init.normal_(self.conv2.bias.data)
        if self.downsample is not None:
            nn.init.xavier_uniform_(self.downsample.weight, gain=np.sqrt(2))
            # nn.init.normal_(self.downsample.weight.data)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)




### Standard Wide & Deep

### PECAD - Single TCN

### PECAD

## Utils

In [0]:
class ArrayDataset(TensorDataset):

    def __init__(self, *arrs):
        super(ArrayDataset, self).__init__()
        # init tensors
        self.tensors = [torch.from_numpy(arr) for arr in arrs]
        assert all(self.tensors[0].size(0) == tensor.size(0) for tensor in self.tensors)

    def data_loader(self, batch_size=128, shuffle=True, num_workers=4):
        return DataLoader(self, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)


In [0]:
from sklearn.metrics import mean_squared_error as mse

def RMSE(output: torch.tensor, target: torch.tensor, std: float, mean: float):
    """
    $$ RMSE = \sqrt{\frac{1}{n}\sum_{t=1}^{n}(\hat{y}_t^2-y_t^2)}$$

    :param output: model predicted tensor
    :param target: target tensor
    :param std: standard deviation
    :param mean: mean
    :return:
    """
    normalized_output = output * std + mean
    normalized_target = target * std + mean

    return torch.sqrt(torch.mean((normalized_output - normalized_target) ** 2))

def val_RMSE(pred_arr, tar_arr, std, mean):
    nomalized_pre_arr = pred_arr * std + mean
    nomalized_test_y = tar_arr * std + mean
    #     print(f"predict array shape: {nomalized_pre_arr.shape}; target array shape: {nomalized_test_y.shape}")
    return np.sqrt(mse(nomalized_pre_arr, nomalized_test_y))


def list2arr(li: list):
    """
    convert list to numpy array
    :param li: list
    :return: converted np array
    """
    arr = np.array(li)
    return arr.reshape(arr.shape[0], arr.shape[1])


def validate(model: nn.Module, val_loader: DataLoader, std, mean):
    """
    validate the test set in the model
    :param model: neural network modal
    :param val_loader: test loader
    :param std: standard deviation
    :param mean: mean value
    :return: RMSE on validation set; loss list on validation set
    """
    # track values
    pred_list, tar_list, loss_list = tuple([] for _ in range(3))
    # L2 loss
    criterion = nn.MSELoss()
    # model evaluation
    model.eval()
    with torch.no_grad():
        for i, (x, y) in enumerate(val_loader):
            x, y = x.to(device), y.to(device)
            result = model(x.float())

            result_list = result.cpu().data.numpy().tolist()
            pred_list.extend(result_list)
            tar_list.extend(y.cpu().data.numpy().tolist())

            loss = criterion(result, y.float()).cpu().data.numpy()
            loss_list.append(loss)

    pred_arr = list2arr(pred_list)
    tar_arr = list2arr(tar_list)
    assert pred_arr.shape == tar_arr.shape

    rmse = val_RMSE(pred_arr, tar_arr, std, mean)
    return rmse, loss_list


def poly_lr_scheduler(optimizer, init_lr, iter, lr_decay_iter=1,
                      max_iter=50, power=0.9):
    """
    Polynomial decay of learning rate
    :param init_lr is base learning rate
    :param iter is a current iteration
    :param lr_decay_iter how frequently decay occurs, default is 1
    :param max_iter is number of maximum iterations
    :param power is a polymomial power

    """
    if iter % lr_decay_iter or iter > max_iter:
        return optimizer

    lr = init_lr*(1 - iter/max_iter)**power
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return lr


def train(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader,
          epochs: int, lr: float, device: torch.device,
          train_std: float, train_mean: float, 
          val_std: float, val_mean: float, criterion=None):
    """
    Train model on train_loader and track the validation value on val_loader.

    :param model: training neural network model
    :param train_loader: train set loader
    :param val_loader: validation set loader
    :param epochs: training epoches
    :param lr: learning rate
    :param train_std: training set's standard deviation
    :param train_mean: training set's average value
    :param val_std: validation set's standard deviation
    :param val_mean: validation set's average value
    :param log: logger object
    :param criterion: loss function (default: L1 loss)
    :return: loss_list, rmse_list, val_loss_list, val_rmse_list
    """
    # loss function
    criterion = nn.L1Loss() if criterion is None else criterion
    # optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.98), eps=1e-9)
    # track loss list
    loss_list, rmse_list, val_loss_list, val_rmse_list = tuple([] for _ in range(4))
    total_steps = len(train_loader)

    for epoch in range(epochs):
        # start training
        model.train()
        poly_lr_scheduler(optimizer, lr, epoch+1)

        for ix, (x, y) in enumerate(train_loader):
            # track starting time
            start_time = time.time()

            # clear the accumulated gradient before each instance
            model.zero_grad()

            # prepare the data and label
            x, y = x.to(device), y.to(device)

            # run the forward pass
            outputs = model(x.float())

            # Compute the loss, gradients, and update the parameters
            loss = criterion(outputs, y.float())

            # back propogation
            loss.backward()
            # update parameters
            optimizer.step()
            # clear gradient
            optimizer.zero_grad()

            # append loss to the loss list
            rmse = RMSE(outputs, y.float(), train_std, train_mean)
            rmse_list.append(rmse)
            loss_list.append(loss)

            # print in every 50 episodes
            if (ix+1) % 50 == 0:
                print(f'Epoch [{epoch + 1}/{epochs}], Step [{ix + 1}/{total_steps}], '
                      f'Time [{time.time() - start_time:.6f} sec], Avg loss: {sum(loss_list[-50:])/50: .4f}, '
                      f'Avg RMSE: {sum(rmse_list[-50:])/50: .4f}')

        # validation
        model.eval()
        print("Validating on the testing set...")
        # TODO: validate crops separately
        val_rmse, val_loss_list = validate(model, val_loader, val_std, val_mean)
        val_avg_loss = np.mean(val_loss_list)
        val_rmse_list.append(val_rmse)
        val_loss_list.append(val_avg_loss)
        print(f"RMSE: {val_rmse: .4f}; avg loss: {val_avg_loss: .4f}")

    return loss_list, rmse_list, val_loss_list, val_rmse_list


In [0]:
def norm_arrays(*arrays, std, mean):
    print(f"std: {std}, mean: {mean}")
    return tuple((arr - mean) / std for arr in arrays)


def save_arrays(**arrs):
    for name, li in arrs.items():
        log.info(f"Saving {data_folder}/{name}.npy")
        np.save(f'{data_folder}/{name}.npy', np.array(li))


def load_numpys(path, files: list):
    return tuple(np.load(f"{path}/{file}.npy") for file in files)

## Training

In [3]:

DAYS = ["4 Days", "6 Days", "9 Days"]
MODELS = ["RF", "GTB", "Attention-LSTM", "TCN", "Standard Wide & Deep", 
          "PECAD - Single TCN", "PECAD"]
CROPS = ['Brinjal', 'Green Chilli', 'Tomato']
LENGTHS = [90, 60, 40]


result = {
    model: {
        day: { crop: None for crop in CROPS } for day in DAYS 
    } for model in MODELS 
}
import pprint as p
p.pprint(result)

{'Attention-LSTM': {'4 Days': {'Brinjal': None,
                               'Green Chilli': None,
                               'Tomato': None},
                    '6 Days': {'Brinjal': None,
                               'Green Chilli': None,
                               'Tomato': None},
                    '9 Days': {'Brinjal': None,
                               'Green Chilli': None,
                               'Tomato': None}},
 'GTB': {'4 Days': {'Brinjal': None, 'Green Chilli': None, 'Tomato': None},
         '6 Days': {'Brinjal': None, 'Green Chilli': None, 'Tomato': None},
         '9 Days': {'Brinjal': None, 'Green Chilli': None, 'Tomato': None}},
 'PECAD': {'4 Days': {'Brinjal': None, 'Green Chilli': None, 'Tomato': None},
           '6 Days': {'Brinjal': None, 'Green Chilli': None, 'Tomato': None},
           '9 Days': {'Brinjal': None, 'Green Chilli': None, 'Tomato': None}},
 'PECAD - Single TCN': {'4 Days': {'Brinjal': None,
                                   '

### DL Model

In [0]:
batch_size = 512

for crop in CROPS:
    for length in LENGTHS:
        day = f"{int(360/length)} Days"
        path = f"./arrays/{crop}/train_{length}_01_test_{length}_01"
        pri_train_x, pri_train_y, vol_train_x, vol_train_y, \
            pri_test_x, pri_test_y, vol_test_x, vol_test_y = \
                load_numpys(path, ["pri_train_x", "pri_train_y", "vol_train_x", "vol_train_y", 
                                "pri_test_x", "pri_test_y", "vol_test_x", "vol_test_y"])

        # normalization
        pri_std, pri_mean = np.std(pri_train_x), np.mean(pri_train_x)
        vol_std, vol_mean = np.std(vol_train_x), np.mean(vol_train_x)
        pri_train_x, pri_train_y, pri_test_x, pri_test_y = \
            norm_arrays(pri_train_x, pri_train_y, pri_test_x, pri_test_y, 
                        std=pri_std, mean=pri_mean)
        vol_train_x, vol_train_y, vol_test_x, vol_test_y = \
            norm_arrays(vol_train_x, vol_train_y, vol_test_x, vol_test_y,
                        std=vol_std, mean=vol_mean)
        print(f"Price std: {pri_std}, Price Mean: {pri_mean}")
        print(f"Volume std: {vol_std}, Volume Mean: {vol_mean}")
        print(f"pri_train_x: {pri_train_x}")
        print(f"price_train_y: {pri_train_y}")
        
        # train_x/train_y
        train_x = np.stack((pri_train_x, vol_train_x), axis=-1)
        train_y = np.copy(pri_train_y)
        print(f"train_x shape: {train_x.shape}, train_y shape: {train_y.shape}")

        # test_x/test_y
        test_x = np.stack((pri_test_x, vol_test_x), axis=-1)
        test_y = np.copy(pri_test_y)

        # train/test loader
        train_loader = ArrayDataset(train_x, train_y).data_loader(batch_size=batch_size)
        test_loader = ArrayDataset(test_x, test_y).data_loader(batch_size=batch_size)

        # Attention-LSTM
        # lstm = LSTM(inputDim=2, hiddenNum=16, outputDim=1, seq_len=length, layerNum=1)
        lstm = AttenLSTM(inputDim=2, hiddenNum=32, outputDim=1, seq_len=length, layerNum=3)
        lstm.to(device)

        _, _, _, val_rmse_list = \
            train(model=lstm, train_loader=train_loader, val_loader=test_loader, 
                epochs=2, lr=0.01, train_std=pri_std, train_mean=pri_mean,
                val_std=pri_std, val_mean=pri_mean, device=device)
        result["Attention-LSTM"][day][crop] = min(val_rmse_list)

        # TCN
        tcn = TCN(input_size=2, output_size=1, num_channels=[16, 8, 2], 
                  input_len=90, output_len=1, kernel_size=4)
        tcn.to(device)
        _, _, _, val_rmse_list = \
            train(model=tcn, train_loader=train_loader, val_loader=test_loader, 
                epochs=20, lr=0.01, train_std=pri_std, train_mean=pri_mean,
                val_std=pri_std, val_mean=pri_mean, device=device)
        result["TCN"][day][crop] = min(val_rmse_list)



### ML Model

In [5]:
from sklearn.ensemble import RandomForestRegressor as RF, \
        GradientBoostingRegressor as GBT

from sklearn.metrics import mean_squared_error as mse

for crop in CROPS:
    for length in LENGTHS:
        day = f"{int(360/length)} Days"
        path = f"./arrays/{crop}/train_{length}_01_test_{length}_01"

        # load data
        pri_train_x, pri_train_y, vol_train_x, vol_train_y, \
            pri_test_x, pri_test_y, vol_test_x, vol_test_y = \
                load_numpys(path, ["pri_train_x", "pri_train_y", "vol_train_x", "vol_train_y", 
                                "pri_test_x", "pri_test_y", "vol_test_x", "vol_test_y"])
        
        # train/test set
        train_x = np.concatenate((pri_train_x, vol_train_x), axis=1)
        train_y = np.copy(pri_train_y)
        test_x = np.concatenate((pri_test_x, vol_test_x), axis=1)
        test_y = np.copy(pri_test_y)

        # random forest
        rf = RF(n_estimators=100, max_depth=5)
        rf.fit(train_x, train_y)
        rf_pred = rf.predict(test_x)
        result["RF"][day][crop] = np.sqrt(mse(test_y, rf_pred))

        # Gradient Boosting Tree
        gbt = GBT(n_estimators=100, max_depth=5)
        gbt.fit(train_x, train_y)
        gbt_pred = gbt.predict(test_x)
        result["GBT"][day][crop] = np.sqrt(mse(test_y, gbt_pred))



NameError: ignored

### Result

In [0]:
p.pprint(result)